# Analisi dei dati raccolti

In questo notebook esplorerai i dati raccolti dal tuo nodo Bitcoin pruned e
memorizzati nel database PostgreSQL. Assicurati di aver popolato il database
(esegui `docker-compose up` e attendi che alcuni blocchi e snapshot vengano
importati) prima di procedere.

Questo esempio include:
- Connessione al database e caricamento in pandas
- Visualizzazione della dimensione del mempool nel tempo
- Calcolo di statistiche di base


In [ ]:
import os
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt

# Parametri di connessione (modifica se necessario)
PGHOST = os.environ.get('PGHOST', 'localhost')
PGUSER = os.environ.get('PGUSER', 'postgres')
PGPASSWORD = os.environ.get('PGPASSWORD', 'postgres')
PGDATABASE = os.environ.get('PGDATABASE', 'blockchain')

conn = psycopg2.connect(host=PGHOST, user=PGUSER, password=PGPASSWORD, dbname=PGDATABASE)


## Evoluzione del mempool

In [ ]:
# Carica i dati del mempool_snapshot
df_mempool = pd.read_sql('SELECT ts, total_tx, total_size FROM mempool_snapshot ORDER BY ts', conn)

# Mostra le prime righe e statistiche base
print(df_mempool.head())
print(df_mempool.describe())

# Plot dimensione totale del mempool (bytes) nel tempo
plt.figure()
df_mempool.set_index('ts')['total_size'].plot(kind='line')
plt.xlabel('Timestamp')
plt.ylabel('Dimensione mempool (byte)')
plt.title('Dimensione del mempool nel tempo')
plt.show()


## Analisi delle transazioni

In [ ]:
# Carica le transazioni e le euristiche
df_tx = pd.read_sql('SELECT b.block_height, b.ts AS block_time, b.feerate, h.is_rbf, h.equal_output, h.coinjoin_score FROM tx_basic b JOIN tx_heuristics h ON b.txid = h.txid', conn)

# Esempio: Distribuzione dei feerate
plt.figure()
df_tx['feerate'].dropna().hist(bins=50)
plt.xlabel('Feerate (sat/vB)')
plt.ylabel('Conteggio')
plt.title('Distribuzione dei feerate')
plt.show()

# Percentuale di transazioni con equal output per blocco
coinjoin_by_block = df_tx.groupby('block_height')['equal_output'].mean()
plt.figure()
coinjoin_by_block.plot()
plt.xlabel('Altezza del blocco')
plt.ylabel('Percentuale con equal output')
plt.title('Percentuale di transazioni coinjoin-like per blocco')
plt.show()


In [ ]:
conn.close()